In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import confusion_matrix
import cv2 
import glob
import matplotlib.pyplot as plt
import tensorflow as tf 
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Dropout,GlobalAveragePooling2D

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
import os
#print(os.listdir("../input/chest_xray/chest_xray/train/NORMAL"))
train_dir_normal = "C:\\Users\\user\\Desktop\\Project Files\\DLModel\\chest_xray\\train\\NORMAL\\*.jpeg"
train_dir_pneumonia = "C:\\Users\\user\\Desktop\\Project Files\\DLModel\\chest_xray\\train\\PNEUMONIA\\*.jpeg"
val_dir_normal = "C:\\Users\\user\\Desktop\\Project Files\\DLModel\\chest_xray\\val\\NORMAL\\*.jpeg"
val_dir_pneumonia = "C:\\Users\\user\\Desktop\\Project Files\\DLModel\\chest_xray\\val\\PNEUMONIA\\*.jpeg"
test_dir_normal = "C:\\Users\\user\\Desktop\\Project Files\\DLModel\\chest_xray\\test\\NORMAL\\*.jpeg"
test_dir_pneumonia = "C:\\Users\\user\\Desktop\\Project Files\\DLModel\\chest_xray\\test\\PNEUMONIA\\*.jpeg"
# Any results you write to the current directory are saved as output.
%matplotlib inline

In [6]:
def shuffle_in_unison(a,b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [7]:
def convert_to_one_hot(vec, num):
    Y = np.eye(num)[vec.reshape(-1)].T
    return Y

In [8]:
def load_data():
    """A function to load the whole dataset and the labels"""
    train_data = []
    val_data = []
    test_data = []

    train_labels = [] 
    val_labels = []
    test_labels = []



    
    train_files_normal = glob.glob(train_dir_normal) 
    train_files_pneumonia = glob.glob(train_dir_pneumonia)
    val_files_normal = glob.glob(val_dir_normal) 
    val_files_pneumonia = glob.glob(val_dir_pneumonia)

    test_files_normal = glob.glob(test_dir_normal) 
    test_files_pneumonia = glob.glob(test_dir_pneumonia)




    for num, file in enumerate(train_files_normal + train_files_pneumonia):
        img = cv2.imread(file, 1) 
        img = cv2.resize(img, (150,150))
        train_data.append(img)
        if(num+1 <= 1341):
            train_labels.append(0)
        else:
            train_labels.append(1)
    
    for num, file in enumerate(val_files_normal + val_files_pneumonia):
        img = cv2.imread(file, 1)
        img = cv2.resize(img, (150,150))
        val_data.append(img)
        if(num+1 <= 8):
            val_labels.append(0)
        else:
            val_labels.append(1)
    
    for num, file in enumerate(test_files_normal + test_files_pneumonia):
        img = cv2.imread(file, 1)
        img = cv2.resize(img, (150,150))
        test_data.append(img)
        if(num+1 <= 234):
            test_labels.append(0)
        else:
            test_labels.append(1)
    
    
    X_train = np.array(train_data)
    X_val  = np.array(val_data)
    X_test = np.array(test_data)
    
    Y_train = np.array(train_labels)
    Y_val = np.array(val_labels)
    Y_test = np.array(test_labels) 
    
    X_train, Y_train = shuffle_in_unison(X_train,Y_train)
    X_val, Y_val = shuffle_in_unison(X_val,Y_val)
    X_test, Y_test = shuffle_in_unison(X_test,Y_test)


    
    return X_train, Y_train, X_val, Y_val, X_test, Y_test 

In [9]:
# Loading our dataset to the handlers
X_train,Y_train,X_val,Y_val,X_test,Y_test = load_data()

In [10]:
# Further preparing the data 
X_train = X_train/ 255 
X_val = X_val/ 255 
X_test = X_test/ 255

Y_train = convert_to_one_hot(Y_train, 2).T
Y_val = convert_to_one_hot(Y_val, 2).T
Y_test = convert_to_one_hot(Y_test, 2).T

print("shape of X_train : " + str(X_train.shape))
print("shape of X_val : " + str(X_val.shape))
print("shape of X_test : " + str(X_test.shape))
print("shape of Y_train : " + str(Y_train.shape))
print("shape of Y_val : " + str(Y_val.shape))
print("shape of Y_test : " + str(Y_test.shape))

shape of X_train : (5216, 150, 150, 3)
shape of X_val : (16, 150, 150, 3)
shape of X_test : (624, 150, 150, 3)
shape of Y_train : (5216, 2)
shape of Y_val : (16, 2)
shape of Y_test : (624, 2)


In [11]:
# Now let us import the Vgg16 pretrained model
base_model = applications.VGG16(include_top = False, weights = 'C:\\Users\\user\\Desktop\\conf_mat\\vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5') 



In [12]:
# Looking at all the layers of the model
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [13]:
# Defining the optimizers for updating the parameters
sgd = optimizers.Adam(lr = 1e-4)

In [14]:
# now we build a function to train the model and add a bottleneck model to our pre-trained one 
def Train_model(X_train, Y_train, X_test, Y_test, optimizer, base_model, batch_size = 64, num_epochs = 10):
    
    X = base_model.output
    X = Dropout(0.5)(X)
    X = GlobalAveragePooling2D()(X)
    X = Dense(128, activation='relu')(X)
    X = BatchNormalization()(X)
    X = Dense(2, activation='sigmoid')(X)
    model = Model(inputs = base_model.input, outputs = X)
    
    for layer in base_model.layers:
        layer.trainable = False 
        
    model.compile(optimizer = optimizer,
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])
    
    model.fit(X_train,Y_train,
              batch_size = batch_size,
              epochs = num_epochs,
              validation_data = (X_test,Y_test))
    
    return model 

In [15]:
# Trainig is happening in the cloud,will push once we hit a good accuracy
model = Train_model(X_train, Y_train, X_test, Y_test, sgd, base_model, batch_size = 64, num_epochs = 10)

Train on 5216 samples, validate on 624 samples
Epoch 1/10
 128/5216 [..............................] - ETA: 20:19 - loss: 0.7423 - acc: 0.5781

KeyboardInterrupt: 